<a href="https://colab.research.google.com/github/mukaseevru/ds-school/blob/main/final_project/parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подключение библиотек

In [1]:
# Загрузка необходимых модулей
import numpy as np
import pandas as pd
import datetime as dt
import requests
import re
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from time import sleep
from bs4 import BeautifulSoup

# Настройки

In [2]:
# Таймаут в секундах, если ошибка соединения
timeout = 1
# Debug mode
debug = True
# Настройки по сайтам
sites = {}
# Включить рассылку
sites['send'] = True
# banki.ru
sites['banki'] = {}
sites['banki']['on'] = True
sites['banki']['first_page'] = 1
sites['banki']['last_page'] = 50
sites['banki']['send'] = True
# vtb.ru
sites['vtb'] = {}
sites['vtb']['on'] = True
sites['vtb']['first_page'] = 1
sites['vtb']['last_page'] = 10
sites['vtb']['send'] = True
# alfabank.ru
sites['alfa'] = {}
sites['alfa']['on'] = True
sites['alfa']['first_page'] = 2021
sites['alfa']['last_page'] = 2022
sites['alfa']['send'] = True

In [3]:
sites

{'alfa': {'first_page': 2021, 'last_page': 2022, 'on': True, 'send': True},
 'banki': {'first_page': 1, 'last_page': 50, 'on': True, 'send': True},
 'send': True,
 'vtb': {'first_page': 1, 'last_page': 10, 'on': True, 'send': True}}

# Определение функций

In [4]:
# Определение функции загрузки страницы
def load_page(url):
  '''
  Загрузка веб страницы
  '''
  count = 0
  while True:
    try:
      with requests.Session() as session:
        session.headers['Connection'] = 'keep-alive'
        session.headers['Cache-Control'] = 'max-age=0'
        session.headers['Upgrade-Insecure-Requests'] = '1'
        session.headers['User-Agent'] = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.116 Safari/537.36 OPR/40.0.2308.81'
        session.headers['Accept'] = 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'
        session.headers['DNT'] = '1'
        session.headers['Accept-Encoding'] = 'gzip, deflate, lzma, sdch'
        session.headers['Accept-Language'] = 'ru-RU,ru;q=0.8,en-US;q=0.6,en;q=0.4'
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)
        session.proxies = {
            # 'http': '185.198.189.21:8080',
            # 'https': '185.198.189.21:8080'
            }
        response = session.get(url)
      if response.status_code != 200:
        # log.info("Ошибка, код ответа: %s", response.status)
        print('Ошибка, код ответа: %s', response.status_code)
        sleep(timeout)
        count += 1
        if count > 5:
          return (-1)
        # Попробуем снова на следующей итерации цикла
        continue
      # Если дошли до сюда, значит ошибок не было
      return response
    except ConnectionError:
      # Выводим свое сообщение плюс стек трассы
      # log.exception("Ошибка ConnectionError")
      print('Ошибка ConnectionError')
      sleep(timeout)

# Определение функции очистки текста от эмоджи
def remove_emojis(text):
  '''
  Очистка текста от эмоджи
  '''
  emoj = re.compile("["
      u"\U0001F600-\U0001F64F"  # emoticons
      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
      u"\U0001F680-\U0001F6FF"  # transport & map symbols
      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
      u"\U00002500-\U00002BEF"  # chinese char
      u"\U00002702-\U000027B0"
      u"\U00002702-\U000027B0"
      u"\U000024C2-\U0001F251"
      u"\U0001f926-\U0001f937"
      u"\U00010000-\U0010ffff"
      u"\u2640-\u2642" 
      u"\u2600-\u2B55"
      u"\u200d"
      u"\u23cf"
      u"\u23e9"
      u"\u231a"
      u"\ufe0f"  # dingbats
      u"\u3030"
                    "]+", re.UNICODE)
  return re.sub(emoj, ' ', text)

# Определение функции очистки текста от спецсимволов
def clean_text(text):
  '''
  Очистка текста от спецсимволов
  '''
  if pd.isnull(text):
    return np.nan
  text = remove_emojis(text)
  text = text.replace('\n',' ')\
  .replace('\r',' ')\
  .replace('\t',' ')\
  .replace('\u200b',' ')\
  .replace('•',' ')\
  .replace('<',' ')\
  .replace('>',' ')\
  .strip()
  while '  ' in text:
    text = text.replace('  ', ' ')
  return text

# banki.ru

## Загрузка имеющихся данных

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os
os.getcwd()

'/content'

Чтобы каждый раз не выгружать все новости заново, загрузим уже скачанный ранее датасет

In [7]:
# скачать можно здесь: https://drive.google.com/file/d/1qRgl3SQ-DcFo7HK9WJgnQMq1npJDMOgN/view?usp=sharing
banki = pd.read_csv('drive/MyDrive/Colab Notebooks/sber/news/data/banki.csv',
                   dtype={'id': np.str,
                          'title': np.str,
                          'text': np.str,
                          'url': np.str,
                          'time': np.str,
                          'views': np.int64,
                          'comments': np.int64,
                          'source': np.str
                          })
banki.time = pd.to_datetime(banki.time, format='%Y-%m-%d %H:%M:%S')
banki.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113250 entries, 0 to 113249
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   id        113250 non-null  object        
 1   title     113250 non-null  object        
 2   text      113172 non-null  object        
 3   url       113250 non-null  object        
 4   time      113250 non-null  datetime64[ns]
 5   views     113250 non-null  int64         
 6   comments  113250 non-null  int64         
 7   source    113211 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 6.9+ MB


In [8]:
banki.tail()

,id,title,text,url,time,views,comments,source
113245,10938680,В Центробанке назвали долю сотрудников на «уда...,В дистанционном режиме работает примерно полов...,https://www.banki.ru/news/lenta/?id=10938680,2020-12-15 21:49:00,2666,0,Banki.ru
113246,10938679,АКРА присвоило Росэксимбанку рейтинг «AА(RU)»,Аналитическое кредитное рейтинговое агентство ...,https://www.banki.ru/news/lenta/?id=10938679,2020-12-15 21:42:00,2736,0,Banki.ru
113247,10938678,Тулин: постоянные регуляторные послабления — э...,Банковский сектор будет выходить из режима рег...,https://www.banki.ru/news/lenta/?id=10938678,2020-12-15 21:34:00,2279,1,Banki.ru
113248,10938677,Росстат назвал величину сокращения промпроизво...,Промышленное производство в России в ноябре со...,https://www.banki.ru/news/lenta/?id=10938677,2020-12-15 21:18:00,1801,0,Banki.ru
113249,10938676,Банки заключили с застройщиками кредитные дого...,Российские банки заключили с застройщиками кре...,https://www.banki.ru/news/lenta/?id=10938676,2020-12-15 20:45:00,1868,0,Banki.ru


## Цикл парсинга

Создадим цикл, который будет заходить на сайт и скачивать информацию о новостях. Если новость новая, добавляем ее в датасет. Если новость уже есть в датасете, то обновим информацию о просмотрах

In [9]:
if sites['banki']['on']:
  for page in range(sites['banki']['first_page'], sites['banki']['last_page']):
    count_add_articles = 0
    count_upd_views = 0
    count_upd_comments = 0
    count_upd_source = 0
    url = 'https://www.banki.ru/news/lenta/page' + str(page)
    response = load_page(url)
    if response == -1:
      continue
    soup = BeautifulSoup(response.text, 'lxml')
    articles = soup.find_all('ul', class_='text-list text-list--date text-list--date-inline')[0].find_all('li')
    for article in articles:
      # Кол-во просмотров
      views = 0
      # Кол-во комментариев
      comments = 0
      # Источник новости
      source = ''
      article_url = article.find('a', class_='text-list-link')
      if article_url.get('href')[0] == 'h':
          # print('Error url - {}'.format(article_url.get('href')))
          continue
      article_id = str(article_url.get('href').split('=')[1])
      article_info = article.find_all('span', class_='news__info')
      if len(article_info) > 1:
        source = clean_text(article_info[0].text)
        if article_info[1].text != '':
          views = int(clean_text(article_info[1].text.split('\n\t')[1]))
        if article_info[1].find('span', class_='link-with-icon__icon icon-font icon-bubble-16 icon-font--size_small'):
          comments = int(clean_text(article_info[1].text.split('\n\t')[3]))
      else:
        if article_info[0].text != '':
          views = int(clean_text(article_info[0].text.split('\n\t')[1]))
        if article_info[0].find('span', class_='link-with-icon__icon icon-font icon-bubble-16 icon-font--size_small'):
          comments = int(clean_text(article_info[0].text.split('\n\t')[3]))
      if banki[banki['id'] == article_id].shape[0] == 0:
        url = 'https://www.banki.ru' + article_url.get('href')
        response = load_page(url)
        if response == -1:
          continue
        soup = BeautifulSoup(response.text, 'lxml')
        # Title
        title = soup.find('h1', class_='header-h0')
        # Text
        text = soup.find('article', class_='article-text plain-text markup-inside lenta')
        # Time
        time = soup.find('time')
        banki = banki.append({'id': str(article_id),
                              'title': str(clean_text(title.text)),
                              'text': str(clean_text(text.text)),
                              'url': str(url),
                              'time': pd.to_datetime(time.text, format='%d.%m.%Y %H:%M'),
                              'views': int(views),
                              'comments': int(comments),
                              'source': str(source)}, ignore_index=True)
        count_add_articles += 1
      else:
        if banki.loc[banki['id'] == article_id, 'views'].values[0] != views:
          banki.loc[banki['id'] == article_id, 'views'] = views
          count_upd_views += 1
        if banki.loc[banki['id'] == article_id, 'comments'].values[0] != comments:
          banki.loc[banki['id'] == article_id, 'comments'] = comments
          count_upd_comments += 1
        if banki.loc[banki['id'] == article_id, 'source'].values[0] != source:
          banki.loc[banki['id'] == article_id, 'source'] = source
          count_upd_source += 1
    sites['banki']['first_page'] = page
    if debug:
      print('Page {}: ADD {}, UPD {} views, {} comments, {} source'.format(page, count_add_articles, count_upd_views, count_upd_comments, count_upd_source))

Page 1: ADD 0, UPD 9 views, 1 comments, 0 source
Page 2: ADD 0, UPD 32 views, 0 comments, 0 source
Page 3: ADD 0, UPD 43 views, 0 comments, 0 source
Page 4: ADD 0, UPD 48 views, 0 comments, 0 source
Page 5: ADD 0, UPD 2 views, 0 comments, 0 source
Page 6: ADD 0, UPD 10 views, 0 comments, 0 source
Page 7: ADD 0, UPD 15 views, 0 comments, 1 source
Page 8: ADD 0, UPD 0 views, 0 comments, 0 source
Page 9: ADD 0, UPD 0 views, 0 comments, 0 source
Page 10: ADD 0, UPD 0 views, 0 comments, 0 source
Page 11: ADD 0, UPD 0 views, 0 comments, 0 source
Page 12: ADD 0, UPD 0 views, 0 comments, 0 source
Page 13: ADD 0, UPD 0 views, 0 comments, 0 source
Page 14: ADD 0, UPD 0 views, 0 comments, 0 source
Page 15: ADD 0, UPD 0 views, 0 comments, 0 source
Page 16: ADD 0, UPD 0 views, 0 comments, 0 source
Page 17: ADD 0, UPD 0 views, 0 comments, 0 source
Page 18: ADD 0, UPD 0 views, 0 comments, 0 source
Page 19: ADD 0, UPD 0 views, 0 comments, 0 source
Page 20: ADD 0, UPD 0 views, 0 comments, 0 source
Page

In [10]:
banki.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113250 entries, 0 to 113249
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   id        113250 non-null  object        
 1   title     113250 non-null  object        
 2   text      113172 non-null  object        
 3   url       113250 non-null  object        
 4   time      113250 non-null  datetime64[ns]
 5   views     113250 non-null  int64         
 6   comments  113250 non-null  int64         
 7   source    113212 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 6.9+ MB


## Обработка дубликатов

В цикле есть проверка на повтор новостей, но сделаем дополнительную проверку на случай изменения id новости

In [11]:
banki.id.nunique()

113250

In [12]:
banki.drop_duplicates(subset=['id'], inplace=True, keep='last')
banki.shape

(113250, 8)

In [13]:
banki[banki.duplicated()].shape

(0, 8)

## Выгрузка в csv

Выгрузим весь датасет в файл, чтобы иметь возможность его загрузить в начале алгоритма и из других скриптов

In [14]:
banki.to_csv('drive/MyDrive/Colab Notebooks/sber/news/data/banki.csv', index=False)

In [15]:
banki.head()

,id,title,text,url,time,views,comments,source
0,10940845,"Эксперты рассказали, каким образом мошенникам ...",На сегодняшний день телекоммуникационные систе...,https://www.banki.ru/news/lenta/?id=10940845,2021-02-02 09:09:00,4268,0,Известия
1,10940844,Почти половина работников в России заявили о п...,События 2020 года пагубно сказались на мотивац...,https://www.banki.ru/news/lenta/?id=10940844,2021-02-02 08:58:00,2301,3,РБК
2,10940817,Банк « Уралсиб» сообщил о кадровых перестановках,"Станислав Тывес, заместитель председателя прав...",https://www.banki.ru/news/lenta/?id=10940817,2021-02-02 08:54:00,3239,0,Партнёрский материал
3,10940847,Опрос: москвичи не хотят возвращаться в офисы,"Более половины московских сотрудников, перевед...",https://www.banki.ru/news/lenta/?id=10940847,2021-02-02 08:49:00,2234,4,Banki.ru
4,10940843,В России число преступлений с банковскими карт...,В России в 2020 году количество преступлений с...,https://www.banki.ru/news/lenta/?id=10940843,2021-02-02 08:48:00,2631,2,Известия


Самая просматриваемая статья

In [16]:
banki[banki.views == banki.views.max()]

,id,title,text,url,time,views,comments,source
98142,8421613,"Арестован банкир, подозреваемый в организации ...",Сотрудники правоохранительных органов задержал...,https://www.banki.ru/news/lenta/?id=8421613,2015-11-03 09:09:00,1674117,29,Коммерсант


## Очистка (убрать комментарии)

In [17]:
# banki['title'] = banki.title.apply(clean_text)
# banki['text'] = banki.text.apply(clean_text)

# vtb.ru

Повторим вышереализованный алгоритм для сайта ВТБ

## Загрузка имеющихся данных

In [18]:
# скачать можно здесь: https://drive.google.com/file/d/1pNykrWil_aDGVfpHcz1fXDN-4gglXdf3/view?usp=sharing
vtb = pd.read_csv('drive/MyDrive/Colab Notebooks/sber/news/data/vtb.csv',
                   dtype={'id': np.str,
                          'title': np.str,
                          'text': np.str,
                          'url': np.str,
                          'time': np.str,
                          'views': np.int64,
                          'comments': np.int64,
                          'source': np.str
                          })
vtb.time = pd.to_datetime(vtb.time, format='%Y-%m-%d %H:%M:%S')
vtb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        2891 non-null   object        
 1   title     2889 non-null   object        
 2   text      2889 non-null   object        
 3   url       2891 non-null   object        
 4   time      2891 non-null   datetime64[ns]
 5   views     2891 non-null   int64         
 6   comments  2891 non-null   int64         
 7   source    2891 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 180.8+ KB


## Цикл парсинга

Создадим цикл для парсинга новостей с сайта ВТБ

In [19]:
if sites['vtb']['on']:
  for page in range(sites['vtb']['first_page'], sites['vtb']['last_page']):
    count_add_articles = 0
    url = 'https://vtb.ru/o-banke/press-centr/novosti-i-press-relizy/?page=' + str(page)
    response = load_page(url)
    if response == -1:
      continue
    soup = BeautifulSoup(response.text, 'lxml')
    articles = soup.find_all('div', class_='news-grid__content__item')
    for article in articles:
      # Кол-во просмотров
      views = 0
      # Кол-во комментариев
      comments = 0
      # Источник новости
      source = 'vtb.ru'
      article_url = article.find('a').get('href')
      if article_url == '':
        continue
      article_id = str(article_url.split('/')[6])
      if vtb[vtb['id'] == article_id].shape[0] == 0:
        url = 'https://vtb.ru' + article_url
        response = load_page(url)
        if response == -1:
          continue
        soup = BeautifulSoup(response.text, 'lxml')
        # Title
        title = soup.find('section', class_='common-header').find('h1').text
        # Text
        text = soup.find('div', class_='news-item__main common-text').text
        # Time
        time = soup.find('div', class_='date-block').text.replace('дата','').replace('\n','')
        vtb = vtb.append({'id': str(article_id),
                        'title': str(clean_text(title)),
                        'text': str(clean_text(text)),
                        'url': str(url),
                        'time': pd.to_datetime(time, format='%d.%m.%Y'),
                        'views': int(views),
                        'comments': int(comments),
                        'source': str(source)}, ignore_index=True)
        count_add_articles += 1
    sites['vtb']['first_page'] = page
    if debug:
      print('Page {}: ADD {}'.format(page, count_add_articles))

Page 1: ADD 7
Page 2: ADD 8
Page 3: ADD 3
Page 4: ADD 0
Page 5: ADD 0
Page 6: ADD 0
Page 7: ADD 0
Page 8: ADD 0
Page 9: ADD 0


In [20]:
vtb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2909 entries, 0 to 2908
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        2909 non-null   object        
 1   title     2907 non-null   object        
 2   text      2907 non-null   object        
 3   url       2909 non-null   object        
 4   time      2909 non-null   datetime64[ns]
 5   views     2909 non-null   int64         
 6   comments  2909 non-null   int64         
 7   source    2909 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 181.9+ KB


## Обработка дубликатов

In [21]:
vtb.id.nunique()

2909

In [22]:
vtb.drop_duplicates(subset=['id'], inplace=True, keep='last')
vtb.shape

(2909, 8)

In [23]:
vtb[vtb.duplicated()].shape

(0, 8)

## Выгрузка в csv

In [24]:
vtb.to_csv('drive/MyDrive/Colab Notebooks/sber/news/data/vtb.csv', index=False)

In [25]:
vtb.tail()

,id,title,text,url,time,views,comments,source
2904,2021-02-08-vtb-zapustil-chat-bot-v-telegram,ВТБ запустил чат-бот в Telegram,ВТБ представил клиентам новый чат-бот на платф...,https://vtb.ru/o-banke/press-centr/novosti-i-p...,2021-02-08,0,0,vtb.ru
2905,2021-02-05-vtb-preduprezhdaet-o-novom-vide-mos...,ВТБ предупреждает о новом виде мошенничества в...,В последнее время ВТБ фиксирует рост активност...,https://vtb.ru/o-banke/press-centr/novosti-i-p...,2021-02-05,0,0,vtb.ru
2906,2021-02-05-fond-obligatsiy-vtb-stal-liderom-po...,Фонд облигаций ВТБ стал лидером по доходности ...,ВТБ Фонд Казначейский в январе показал лучшую ...,https://vtb.ru/o-banke/press-centr/novosti-i-p...,2021-02-05,0,0,vtb.ru
2907,2021-02-05-vtb-v-yanvare-rossiyane-na-chetvert...,ВТБ: в январе россияне на четверть увеличили с...,Траты россиян на розничные товары и услуги в я...,https://vtb.ru/o-banke/press-centr/novosti-i-p...,2021-02-05,0,0,vtb.ru
2908,2021-02-01-vtb-nachinaet-prodazhi-subordinirov...,ВТБ начинает продажи субординированных облигац...,С 8 февраля 2021 года стартуют продажи бессроч...,https://vtb.ru/o-banke/press-centr/novosti-i-p...,2021-02-05,0,0,vtb.ru


## Очистка (убрать комментарии)

In [26]:
# vtb['title'] = vtb.title.apply(clean_text)
# vtb['text'] = vtb.text.apply(clean_text)

# alfabank.ru

Повторим вышереализованный алгоритм для сайта Альфабанка

## Загрузка имеющихся данных

In [27]:
# скачать можно здесь: https://drive.google.com/file/d/1k7mKvSyUvAbflRHuK8DlLwAykxCTVABG/view?usp=sharing
alfa = pd.read_csv('drive/MyDrive/Colab Notebooks/sber/news/data/alfa.csv',
                   dtype={'id': np.str,
                          'title': np.str,
                          'text': np.str,
                          'url': np.str,
                          'time': np.str,
                          'views': np.int64,
                          'comments': np.int64,
                          'source': np.str
                          })
alfa.time = pd.to_datetime(alfa.time, format='%Y-%m-%d %H:%M:%S')
alfa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4054 entries, 0 to 4053
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        4054 non-null   object        
 1   title     4054 non-null   object        
 2   text      4032 non-null   object        
 3   url       4054 non-null   object        
 4   time      4054 non-null   datetime64[ns]
 5   views     4054 non-null   int64         
 6   comments  4054 non-null   int64         
 7   source    4054 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 253.5+ KB


## Цикл парсинга

In [28]:
if sites['alfa']['on']:
  url_base = 'https://alfabank.ru'
  for url_year in range(sites['alfa']['first_page'], sites['alfa']['last_page']):
    count_add_articles = 0
    for url_month in range(1, 13):
      if url_month < 10:
        url_month = '0' + str(url_month)
      url = url_base + '/press/news/' + str(url_year) + '/' + str(url_month) + '/'
      response = load_page(url)
      if response == -1:
        continue
      soup = BeautifulSoup(response.content, 'lxml')
      articles = soup.find_all('div', class_='news-item')
      if len(articles) == 0:
        continue
      for article in articles:
        # Кол-во просмотров
        views = 0
        # Кол-во комментариев
        comments = 0
        # Источник новости
        source = 'alfabank.ru'
        article_url = article.find('a').get('href')
        article_id = str(article_url.split('/')[3]) + '_' + str(article_url.split('/')[4]) + '_' + str(article_url.split('/')[5]) + '_' + str(article_url.split('/')[6].split('.')[0])
        if alfa[alfa['id'] == article_id].shape[0] == 0:
          # Title
          title = article.find('div', class_='title').text
          # Time
          time = article.find('div', class_='date').text
          url = url_base + article_url
          response = load_page(url)
          if response == -1:
            continue
          soup = BeautifulSoup(response.content, 'lxml')
          article_text = soup.find('div', class_='news-article').find_all('p')
          # Text
          text = ''
          for text_string in article_text[1:]:
            text += text_string.text
          alfa = alfa.append({'id': str(article_id),
                              'title': str(clean_text(title)),
                              'text': str(clean_text(text)),
                              'url': str(url),
                              'time': pd.to_datetime(time, format='%d.%m.%Y'),
                              'views': int(views),
                              'comments': int(comments),
                              'source': str(source)}, ignore_index=True)
          count_add_articles += 1
    sites['alfa']['first_page'] = url_year
    if debug:
      print('Year {}, ADD {}'.format(url_year, count_add_articles))

Year 2021, ADD 0


In [29]:
alfa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4054 entries, 0 to 4053
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        4054 non-null   object        
 1   title     4054 non-null   object        
 2   text      4032 non-null   object        
 3   url       4054 non-null   object        
 4   time      4054 non-null   datetime64[ns]
 5   views     4054 non-null   int64         
 6   comments  4054 non-null   int64         
 7   source    4054 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 253.5+ KB


## Обработка дубликатов

In [30]:
alfa.id.nunique()

4054

In [31]:
alfa.drop_duplicates(subset=['id'], inplace=True, keep='last')
alfa.shape

(4054, 8)

In [32]:
alfa[alfa.duplicated()].shape

(0, 8)

## Выгрузка в csv

In [33]:
alfa.to_csv('drive/MyDrive/Colab Notebooks/sber/news/data/alfa.csv', index=False)

In [34]:
alfa.tail()

,id,title,text,url,time,views,comments,source
4049,2021_2_1_74267,Пополнение карт Альфа-Банка в салонах «Связной...,Вносить деньги на карты Альфа-Банка теперь мож...,https://alfabank.ru/press/news/2021/2/1/74267....,2021-02-01,0,0,alfabank.ru
4050,2021_2_3_74431,"Альфа-Банк выдаст 1,1 млрд рублей на реконстру...","Альфа-Банк выдаст 1,1 млрд рублей на реконстру...",https://alfabank.ru/press/news/2021/2/3/74431....,2021-02-03,0,0,alfabank.ru
4051,2021_2_2_74380,Альфа-Групп провела чемпионат для дата-сайенти...,"Альфа-Банк, X5 Retail Group, Билайн и АльфаСтр...",https://alfabank.ru/press/news/2021/2/2/74380....,2021-02-02,0,0,alfabank.ru
4052,2021_2_10_74857,А-Клуб стал лучшим в рейтинге private banking ...,Международный журнал Euromoney признал А-Клуб ...,https://alfabank.ru/press/news/2021/2/10/74857...,2021-02-10,0,0,alfabank.ru
4053,2021_2_12_74984,При поддержке Альфа-Банка завершен второй этап...,При поддержке Альфа-Банка в Кемеровской област...,https://alfabank.ru/press/news/2021/2/12/74984...,2021-02-12,0,0,alfabank.ru


## Очистка (убрать комментарии)

In [35]:
# alfa['title'] = alfa.title.apply(clean_text)
# alfa['text'] = alfa.text.apply(clean_text)

# Выгрузка новостей для рассылки

Реализуем простейший алгоритм подбора новостей для рассылки коллегам

In [36]:
banki.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113250 entries, 0 to 113249
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   id        113250 non-null  object        
 1   title     113250 non-null  object        
 2   text      113172 non-null  object        
 3   url       113250 non-null  object        
 4   time      113250 non-null  datetime64[ns]
 5   views     113250 non-null  int64         
 6   comments  113250 non-null  int64         
 7   source    113212 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 7.8+ MB


In [37]:
# номер текущей недели
number_of_week = dt.datetime.now().date().isocalendar()[1]

In [38]:
if sites['send']:
  with pd.ExcelWriter('drive/MyDrive/Colab Notebooks/sber/news/data/send.xlsx') as writer:  
    banki_send = pd.DataFrame(columns=banki.columns)
    if sites['banki']['send']:
      for i in range(1, 8):
        banki_send = banki_send.append(banki[(banki.time.dt.isocalendar().year == 2021) &\
                                             (banki.time.dt.isocalendar().week == number_of_week) &\
                                             (banki.time.dt.isocalendar().day == i)].sort_values(by='views', ascending=False).head(3))
      display(banki_send.head())
      banki_send.to_excel(writer, sheet_name='banki')
    elif sites['vtb']['send']:
      vtb_send = vtb[(vtb.time.dt.isocalendar().year == 2021) & (vtb.time.dt.isocalendar().week == number_of_week)].sort_values(by='time', ascending=True)
      display(vtb_send.head())
      vtb_send.to_excel(writer, sheet_name='vtb')
    elif sites['alfa']['send']:
      alfa_send = alfa[(alfa.time.dt.isocalendar().year == 2021) & (alfa.time.dt.isocalendar().week == number_of_week)].sort_values(by='time', ascending=True)
      display(alfa_send.head())
      alfa_send.to_excel(writer, sheet_name='alfa')
    else:
      banki_send.to_excel(writer, sheet_name='clean')

,id,title,text,url,time,views,comments,source
111000,10941147,Банк «Открытие» предлагает сниженную ставку по...,Банк «Открытие» обновил линейку кредитов налич...,https://www.banki.ru/news/lenta/?id=10941147,2021-02-08 10:44:00,16372,3,Banki.ru
111003,10941138,Газпромбанк проводит акцию по кредитам наличными,Газпромбанк проводит акцию и в течение недели ...,https://www.banki.ru/news/lenta/?id=10941138,2021-02-08 10:05:00,13024,0,Banki.ru
111005,10941139,8 февраля. О чем пишут газеты,Экономисты конгресса США указали на проблему п...,https://www.banki.ru/news/lenta/?id=10941139,2021-02-08 09:29:00,7174,0,
112961,10941243,"ЦБ оштрафовал банк «Открытие», «АльфаСтрахован...","ЦБ привлек банк «Открытие», «АльфаСтрахование»...",https://www.banki.ru/news/lenta/?id=10941243,2021-02-09 15:21:00,8699,0,Banki.ru
112972,10941230,СберБанк запустил рефинансирование ипотеки на ...,СберБанк запустил рефинансирование выданных др...,https://www.banki.ru/news/lenta/?id=10941230,2021-02-09 12:02:00,7792,1,Banki.ru


# Вывод

Данный скрипт позволяет осуществлять парсинг 3 источников (банки, ВТБ, альфабанк), очищать данные и делать выборку новостей для рассылки